<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="../media/inria-25-alpha.png" /></span>
</div>

# quand ça coince

*décrit les raisons pour lesquelles git refuse parfois de travailler, et comment contourner notamment avec le stash*